In [ ]:
import findspark

findspark.init()

from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc
import json
import pickle

sc = SparkContext()
ssc = StreamingContext(sc, 10 )
sqlContext = SQLContext(sc)

socket_stream = ssc.socketTextStream("127.0.0.1", 5555)

lines = socket_stream.window( 20,20 )




from collections import namedtuple
fields = ("text", "location","source" )
Tweet = namedtuple( 'Tweet', fields )

# lines.map(lambda x:x.toString).foreachRDD(lambda rddRaw : rddRaw.toDF()).limit(10).registerTempTable("tweets1")


( lines.map( lambda text: text.split( "~@" ) ) #Splits to a list
  .foreachRDD( 
      lambda rdd: rdd.toDF().registerTempTable("tweets") 
  )) # Registers to a table.


#lines.pprint()

In [ ]:
import time
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns
import pandas
# Only works for Jupyter Notebooks!
%matplotlib inline

ssc.start()


In [ ]:
count = 0
while count < 1:
    
    time.sleep( 5 )
    
    top_10_source = sqlContext.sql('select source,count(*) as count from(select regexp_extract(tweets._3,\'Android|Twitter Web App|iPhone|Facebook|HubSpot|iPad|EcoInternet3|Echofon\',0) as source from tweets) where nullif(source,"") is not null group by source' )
    top_10_df = top_10_source.toPandas()
    display.clear_output(wait=True)
    plt.figure( figsize = ( 10, 8 ) )
    sns.barplot( x="count", y="source", data=top_10_df)
    plt.show()
    
    count = count + 1

In [ ]:
count = 0
while count < 1:
    
    time.sleep( 5 )
    top_10_source = sqlContext.sql( 'select tweets._2 as location,count(*) as count from tweets where nullif(tweets._2,"") is not null and tweets._2 not like "%None%" group by tweets._2' )
    top_10_df = top_10_source.toPandas()
    display.clear_output(wait=True)
    plt.figure( figsize = ( 10, 8 ) )
    sns.barplot( x="count", y="location", data=top_10_df)
    plt.show()
    
    count = count + 1

In [ ]:
ssc.stop()